In [1]:
# Import Splinter and BeautifulSoup
from splinter import Browser
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import time

In [2]:
#from webdriver_manager.chrome import ChromeDriverManager
#executable_path = {'executable_path': ChromeDriverManager().install()}
#print(executable_path)

In [3]:
# Alternate to chrome executable path using selenium Service
from selenium.webdriver.chrome.service import Service

In [4]:
my_service = Service(executable_path=r'/Users/cassie/.wdm/drivers/chromedriver/mac64/117.0.5938.92/chromedriver-mac-x64/chromedriver')

In [5]:
browser = Browser('chrome', service=my_service)



In [6]:
url = 'https://www.planecrashinfo.com/database.htm'
browser.visit(url)

In [10]:
df = pd.DataFrame(columns=['date', 'time', 'location','operator','aboard','fatalities'])
html = browser.html
soup = BeautifulSoup(html, 'html.parser')
year_list = browser.links.find_by_partial_href('/')
year = 1920
time.sleep(1) #tried to slow it down so it didnt crash, dont think this is the right location cuz it didnt work
for x in range(year,2023):
    url_text = 'https://www.planecrashinfo.com/'+str(x)+'/'+str(x)+'.htm'
    print(url_text)
    browser.visit(url_text)
    event_list = browser.links.find_by_partial_href('-')
    for event in event_list:
        event.click()
        html = browser.html
        soup = BeautifulSoup(html, 'html.parser')
        date = soup.find("td", string="Date:").find_next('td').text
        time = soup.find("td", string="Time:").find_next('td').text
        location = soup.find("td", string="Location:").find_next('td').text
        operator = soup.find("td", string="Operator:").find_next('td').text
        aboard_temp = soup.find("td", string="Aboard:").find_next('td').text
        aboard = aboard_temp.split(' ',1)[0]
        fatalities_temp = soup.find("td", string="Fatalities:").find_next('td').text
        fatalities = fatalities_temp.split(' ',1)[0]
        new_row = {'date':date,'time':time,'location':location,'operator':operator,'aboard':aboard,'fatalities':fatalities}
        print(new_row)
        new_row_df = pd.DataFrame([new_row])
        df = pd.concat([df, new_row_df])
        browser.back()
   


AttributeError: 'str' object has no attribute 'sleep'

In [ ]:
df


In [11]:
from geopy.geocoders import Nominatim

In [12]:
df['location'] = df['location'].astype(str)
print (type(df['location'].iloc[0]))
df = df.reset_index(drop=True)

<class 'str'>


In [13]:
geolocator = Nominatim(timeout=10, user_agent = "jimmy")
location = geolocator.geocode('Victoria, Capital Regional District, British Columbia, Canada')
print(location)
print((location.latitude, location.longitude))
#to test for append accuracy

Victoria, Capital Regional District, British Columbia, Canada
(48.4283182, -123.3649533)


In [18]:
#lats = np.array([])
#lons = np.array([])

locations = []

geolocator = Nominatim(timeout=10, user_agent = "jimmy")
for index, row in df.iterrows():
    location = geolocator.geocode(row['location'])
    print(location)   #got it to itterate thru correctly but getting stuck on nones. using this as reference and copied below
    #if location != "none":                    
        #lats = np.append(location.latitude, lats)
        #lons = np.append(location.longitude, lons)
    try:
        location_dict = {
            "location": location,
            "lat": location.latitude,
            "lon": location.longitude
        }
        #print(location.latitude)
        #print(location.longitude)
        #print(location)
        locations.append(location_dict)
        print("---")
    except AttributeError:
        print(f'Skipping: {location}')
        continue
    #else:
    #lats = np.append(location.latitude, '0')
    #lons = np.append(location.longitude, "0")

North Fort Myer Drive, Rosslyn, Arlington, Arlington County, Virginia, 22209, United States
---
Juvisy-sur-Orge, Palaiseau, Essonne, Île-de-France, France métropolitaine, 91260, France
---
Atlantic City, Atlantic County, New Jersey, 08401, United States
---
Victoria, Capital Regional District, British Columbia, Canada
---
None
Skipping: None
None
Skipping: None
Tienen, Leuven, Vlaams-Brabant, Vlaanderen, 3300, België / Belgique / Belgien
---
None
Skipping: None
None
Skipping: None
Billericay, Basildon, Essex, England, United Kingdom
---
Potters Bar, Hertsmere, Hertfordshire, England, EN6 1QG, United Kingdom
---
Mainz, Rheinland-Pfalz, Deutschland
---
None
Skipping: None
None
Skipping: None
None
Skipping: None
None
Skipping: None
None
Skipping: None
None
Skipping: None
Brainy Cut Off, Hazelvale, Shire Of Denmark, Western Australia, Australia
---
None
Skipping: None
None
Skipping: None
None
Skipping: None
None
Skipping: None
Elizabeth, Union County, New Jersey, United States
---
Clevelan

Radevormwald, Oberbergischer Kreis, Nordrhein-Westfalen, 42477, Deutschland
---
Frankfurt am Main, Hessen, Deutschland
---
Lebo, Coffey County, Kansas, United States
---
None
Skipping: None
Purley, London Borough of Croydon, London, Greater London, England, CR8 2FQ, United Kingdom
---
Charlois, Rotterdam, Zuid-Holland, Nederland, Waalhaven
---
None
Skipping: None
Toul, Meurthe-et-Moselle, Grand Est, France métropolitaine, 54200, France
---
Adelaide Hills Council, South Australia, Australia
---
Pocatello, Bannock County, Idaho, United States
---
None
Skipping: None
Girona, Gironès, Girona, Catalunya, España
---
Polk, Venango County, Pennsylvania, United States
---
Valence, Drôme, Auvergne-Rhône-Alpes, France métropolitaine, 26000, France
---
None
Skipping: None
Edgerton, St. Joseph Township, Williams County, Ohio, United States
---
Edgerton, St. Joseph Township, Williams County, Ohio, United States
---
Bristol Township, Trumbull County, Ohio, 44402, United States
---
Spur, Dickens Count

In [19]:
print(locations)

[{'location': Location(North Fort Myer Drive, Rosslyn, Arlington, Arlington County, Virginia, 22209, United States, (38.8967386, -77.0724024, 0.0)), 'lat': 38.8967386, 'lon': -77.0724024}, {'location': Location(Juvisy-sur-Orge, Palaiseau, Essonne, Île-de-France, France métropolitaine, 91260, France, (48.6904063, 2.373809, 0.0)), 'lat': 48.6904063, 'lon': 2.373809}, {'location': Location(Atlantic City, Atlantic County, New Jersey, 08401, United States, (39.3642852, -74.4229351, 0.0)), 'lat': 39.3642852, 'lon': -74.4229351}, {'location': Location(Victoria, Capital Regional District, British Columbia, Canada, (48.4283182, -123.3649533, 0.0)), 'lat': 48.4283182, 'lon': -123.3649533}, {'location': Location(Tienen, Leuven, Vlaams-Brabant, Vlaanderen, 3300, België / Belgique / Belgien, (50.8074825, 4.9371775, 0.0)), 'lat': 50.8074825, 'lon': 4.9371775}, {'location': Location(Billericay, Basildon, Essex, England, United Kingdom, (51.6284395, 0.4218142, 0.0)), 'lat': 51.6284395, 'lon': 0.421814

In [20]:
new_df = pd.DataFrame(locations)
new_df

,location,lat,lon
0,"(North Fort Myer Drive, Rosslyn, Arlington, Ar...",38.896739,-77.072402
1,"(Juvisy-sur-Orge, Palaiseau, Essonne, Île-de-F...",48.690406,2.373809
2,"(Atlantic City, Atlantic County, New Jersey, 0...",39.364285,-74.422935
3,"(Victoria, Capital Regional District, British ...",48.428318,-123.364953
4,"(Tienen, Leuven, Vlaams-Brabant, Vlaanderen, 3...",50.807482,4.937177
...,...,...,...
137,"(Newark, Essex County, New Jersey, United Stat...",40.735657,-74.172367
138,"(San Diego, San Diego County, California, Unit...",32.717420,-117.162773
139,"(Tampico, Tamaulipas, 89080, México, (22.21576...",22.215762,-97.857771
140,"(Tököl, Szigetszentmiklósi járás, Pest vármegy...",47.322679,18.963243


In [ ]:
#attempting to get lats and lons
# lats = np.array([])
# lons = np.array([])
# geolocator = Nominatim(timeout=10, user_agent = "jimmy")
# for row in df['location']:
#     locations = geolocator.geocode(row)
#     if location is None:
#         lats = np.append(lats, 'None')    ##its skipping this section when hitting a null value, 
#         lons = np.append(lons, 'None')    ##reading it as doesnt exist rather than none, even tho the output is none directly above
#     else:
#         lats = np.append(lats, locations.latitude)
#         lons = np.append(lons, locations.longitude)

In [ ]:
#lats


In [ ]:
# df['latitude']= lats
# df['longitude'] = lons
#all these titles are all keywords in sql, maybe unnessesary but changing for possible ease of use
#df.rename(columns = {'date':'date_','time':'time_','location':'location_','operator':'operator_'}, inplace = True)
#index will be crash id as main key
#df = df.reset_index(drop=True)

In [ ]:
#df.to_csv('plane_crash_data', encoding='utf-8', index=False)
#saved a csv becasue i got the biggest set of data before it crashed. if someone gets a full or bigger set be sure to save it. 

In [15]:
new_df.to_csv('plane_crash_data_new', encoding='utf-8', index=False)

NameError: name 'new_df' is not defined